In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# !pip install bs_ds
# !pip install fake_useragent
# !pip install lxml
# ! pip install scrapy
# ! pip install scrapy_user_agents
# ! ls ./drive/MyDrive/sharif/Spider
# ! pwd

In [3]:
import os
import pandas as pd
import re
import glob

In [4]:
os.chdir('./drive/MyDrive/sharif/Spider/ipython(guide)')
os.listdir()

['step_by_step_2_Vectorizing.ipynb',
 'clf_SVC.joblib',
 'clf_nb.joblib',
 'clf_mtf.joblib',
 'knn_evaluation.txt',
 'neigh_mtf.joblib',
 'step_by_step_3.ipynb',
 'step_by_step_1_Collecting_Preprocessing_FreqDist.ipynb']

In [ ]:
import os
import pandas as pd
import glob

### Making dataset from crawled .csv files




In [ ]:
final_csv_file = pd.DataFrame(columns=['body', 'tag', 'title'])
# final_csv_file = pd.read_csv('dataset.csv')
csv_file_path = '/body_tag_title.csv'
file_name = glob.glob('../spider_*') 
tags_list = pd.read_excel('../Tags.xlsx').values
print(file_name)
! ls ../spider_1_1

['../spider_1_1', '../spider_1_3', '../spider_1_0', '../spider_1_2', '../spider_1_4', '../spider_1_6']
body_tag_title.csv  scrapy.cfg	spider_1_1


In [ ]:
for file_n in file_name:
  matrix_main = pd.read_csv(file_n + csv_file_path)
  print(file_n, ' : ', len(matrix_main))
  for i in range(len(matrix_main)):
    matrix_main['tag'][i] = re.sub('Jib:|active|صفحه[\d]+|صفحه [\d]+|,\'active\'|,\'active\',|\'active\',|page=2|<span>|</span>|tag','',matrix_main['tag'][i])
    if matrix_main['tag'][i]=='':
      matrix_main['tag'][i] = None
  matrix_main = matrix_main.dropna(axis=0,inplace=False).reset_index(drop=True)

  counter = 0
  max_counter = 5
  rows_per_iter = 1500
  first_len = len(matrix_main)
  while True:

    counter += 1
    print(' counter: ', counter)
    temp_matrix = pd.DataFrame(columns=['body', 'tag', 'title'])
    pre_len = len(matrix_main)
    for i in range(0, 1+len(matrix_main) // rows_per_iter):
        loop_matrix = matrix_main.loc[i*rows_per_iter:min(first_len,(i+1)*rows_per_iter),:]
        loop_matrix = loop_matrix.dropna(axis=0).reset_index(drop=True)
        for title in loop_matrix['title']:
            indexes = loop_matrix.loc[loop_matrix['title'] == title].index
            if len(indexes) > 0:
                tags = set(loop_matrix.loc[loop_matrix['title'] == title]['tag'])
                tags = re.sub('\{|\}', '', str(tags))
                tags = re.sub('[\']+|[\"]+', '\'', tags)
                loop_matrix.loc[indexes[0], 'title'] = str(title).strip()
                loop_matrix.loc[indexes[0], 'tag'] = tags
                loop_matrix = loop_matrix.drop(axis=0, index=indexes[1:]).reset_index(drop=True)
        temp_matrix = temp_matrix.append(loop_matrix)
        print('  - loop:', len(loop_matrix))
    if len(temp_matrix) > 0:
        matrix_main = temp_matrix.sample(frac=1).reset_index(drop=True)
    print(' - per count: ', len(matrix_main))
    if (counter > max_counter) or (len(matrix_main) == pre_len):
        break
  final_csv_file = final_csv_file.append(matrix_main)
final_csv_file.loc[:,['body','tag']].to_csv('../Data/unpreprocessed_dataset.csv', index=False)

../spider_1_1  :  12745
 counter:  1
  - loop: 980
  - loop: 246
 - per count:  1226
 counter:  2
  - loop: 1088
 - per count:  1088
 counter:  3
  - loop: 1088
 - per count:  1088
../spider_1_3  :  9112
 counter:  1
  - loop: 1054
  - loop: 1180
  - loop: 1264
  - loop: 1108
  - loop: 1067
  - loop: 1002
  - loop: 33
 - per count:  6708
 counter:  2
  - loop: 1178
  - loop: 1154
  - loop: 1188
  - loop: 1166
  - loop: 623
 - per count:  5309
 counter:  3
  - loop: 1222
  - loop: 1220
  - loop: 1225
  - loop: 720
 - per count:  4387
 counter:  4
  - loop: 1264
  - loop: 1281
  - loop: 1180
 - per count:  3725
 counter:  5
  - loop: 1326
  - loop: 1321
  - loop: 680
 - per count:  3327
 counter:  6
  - loop: 1355
  - loop: 1356
  - loop: 322
 - per count:  3033
../spider_1_0  :  25506
 counter:  1
  - loop: 1498
  - loop: 1500
  - loop: 1468
  - loop: 1488
  - loop: 1491
  - loop: 1442
  - loop: 1492
  - loop: 1475
  - loop: 1455
  - loop: 1486
  - loop: 1496
  - loop: 1439
  - loop: 14

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


 counter:  1
  - loop: 1182
  - loop: 1140
  - loop: 169
 - per count:  2491
 counter:  2
  - loop: 1253
  - loop: 883
 - per count:  2136
 counter:  3
  - loop: 1366
  - loop: 614
 - per count:  1980
 counter:  4
  - loop: 1438
  - loop: 471
 - per count:  1909
 counter:  5
  - loop: 1470
  - loop: 407
 - per count:  1877
 counter:  6
  - loop: 1490
  - loop: 376
 - per count:  1866


In [ ]:
len(final_csv_file)

35667

## Frequency of Tags

In [ ]:
Tags=[]
file = pd.read_excel('../Tags.xlsx')
data = pd.read_csv('../Data/unpreprocessed_dataset.csv')
for line in file.iloc[:,0]:
    if len(line) != 0:
        Tags.append(line)
frequency=pd.DataFrame(columns=["tag","number"])
frequency["tag"]=Tags
frequency["number"]=[0]*len(Tags)


In [ ]:
for i in range(len(data)):
    tag=data.loc[i,"tag"].replace("'","").split(",")
    for t in tag:
        ind = frequency.query("tag=="+"'"+t.strip()+"'").index
        frequency.loc[ind,"number"]+=1


In [ ]:
frequency.to_csv("../Data/frequency.csv", index=False)

In [ ]:
print(frequency)

             tag  number
0         پایتون    1126
1        اندروید     722
2   جاوا اسکریپت    1136
3     هوش مصنوعی     970
4         ری اکت     405
..           ...     ...
74           ios     235
75      توسعه وب    1099
76      اپلیکیشن     738
77             c    3735
78        vscode      81

[79 rows x 2 columns]


In [ ]:
sum(frequency['number'][:])

60789

## Preprocessing

In [ ]:
! pip install hazm

     |████████████████████████████████| 317kB 5.6MB/s 
     |████████████████████████████████| 235kB 17.7MB/s 
     |████████████████████████████████| 1.4MB 12.3MB/s 
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp36-cp36m-linux_x86_64.whl size=154314 sha256=ba94feca3dbf9eaf3883f117447ae35dbadc588da4486ed42c35291eb10b2008
  Stored in directory: /root/.cache/pip/wheels/66/15/54/4510dce8bb958b1cdd2c47425cbd1e1eecc0480ac9bb1fb9ab
  Created wheel for nltk: filename=nltk-3.3-cp36-none-any.whl size=1394470 sha256=072b79edbf30d9773a9666bc78d6742333ed8b92cff2459b2e461a57177041ef
  Stored in directory: /root/.cache/pip/wheels/d1/ab/40/3bceea46922767e42986aef7606a600538ca80de6062dc266c
Successfully built libwapiti nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [ ]:
from hazm import *
normalizer=Normalizer()
stemmer=Stemmer()
lemmatizer=Lemmatizer()

def Normalizer(text):

    # normalizer = Normalizer()
    normalized = normalizer.normalize(text)

    return normalized


def SentenceSplitter_Tokenizer(text):

    Tokens=list(word_tokenize(text))

    return Tokens

def Stemmer(token):

    stemming=[]
    for word in token:
        stemming.append(lemmatizer.lemmatize(word))

    return stemming

def RemoveStopwords_Punc(token):

    with open('../Preprocessing/stopwords') as f:
        sw = [line.rstrip().replace(" ","").replace("\u200c","") for line in f]
    # print(sw)

    punctuation = '!"#$%&\'()*+–,-./:;<=>?@[\\]^_`{|}~،؟«؛'
    result=[]
    for word in token:
        for s in punctuation:
            if s in word:
                word=word.replace(s,"")
            # print(word)
        if word.replace(" ","").replace("\u200c","") in sw:
            continue

        result.append(word)

    # print(result)
    return result

def EmojiRemoving(text):

    emoj = re.compile("["
                      u"\U0001F600-\U0001F64F"  # emoticons
                      u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                      u"\U0001F680-\U0001F6FF"  # transport & map symbols
                      u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                      u"\U00002500-\U00002BEF"  # chinese char
                      u"\U00002702-\U000027B0"
                      u"\U00002702-\U000027B0"
                      u"\U000024C2-\U0001F251"
                      u"\U0001f926-\U0001f937"
                      u"\U00010000-\U0010ffff"
                      u"\u2640-\u2642"
                      u"\u2600-\u2B55"
                      u"\u200d"
                      u"\u23cf"
                      u"\u23e9"
                      u"\u231a"
                      u"\ufe0f"  # dingbats
                      u"\u3030"
                      "]+", re.UNICODE)
    text= re.sub(emoj, '', text)

    return text


In [ ]:
data = pd.read_csv('../Data/unpreprocessed_dataset.csv')
for i in range(len(data)):
    text=data.loc[i,"body"]
    # print(text)
    # remove half-space
    text = text.replace("\u200c", " ")
    text = text.replace("های ", " ها ی ")
    text = text.replace("ها ", " ها ")
    # print(text)
    text=EmojiRemoving(text)
    normalized = Normalizer(text)
    # print("norm",normalized)
    split = SentenceSplitter_Tokenizer(normalized)
    # print(split)
    tokens = RemoveStopwords_Punc(split)
    # print("tokens:",tokens)
    # stemming = Stemmer(tokens)
    # print("stemm", stemming)
    tokens = [t for t in tokens if len(t)>2]
    data.loc[i, "body"]=str(tokens)
    
    if i%100==0:
      print("Record: up to "+str(i)+" is processed.")

Record: up to 0 is processed.
Record: up to 100 is processed.
Record: up to 200 is processed.
Record: up to 300 is processed.
Record: up to 400 is processed.
Record: up to 500 is processed.
Record: up to 600 is processed.
Record: up to 700 is processed.
Record: up to 800 is processed.
Record: up to 900 is processed.
Record: up to 1000 is processed.
Record: up to 1100 is processed.
Record: up to 1200 is processed.
Record: up to 1300 is processed.
Record: up to 1400 is processed.
Record: up to 1500 is processed.
Record: up to 1600 is processed.
Record: up to 1700 is processed.
Record: up to 1800 is processed.
Record: up to 1900 is processed.
Record: up to 2000 is processed.
Record: up to 2100 is processed.
Record: up to 2200 is processed.
Record: up to 2300 is processed.
Record: up to 2400 is processed.
Record: up to 2500 is processed.
Record: up to 2600 is processed.
Record: up to 2700 is processed.
Record: up to 2800 is processed.
Record: up to 2900 is processed.
Record: up to 3000 is 

In [ ]:
data.to_csv("../Data/preprocessed.csv", index=False)

In [ ]:
data.head()

,body,tag
0,"['جزو', 'دولوپرهایی', 'سال', 'کدنویسی', 'کسب',...",'' مهارت''
1,"['دولوپر\u200cها', 'ابزار\u200cها', 'مختلفی', ...",'' کاتلین''
2,"['می\u200cشناسید', 'جزو', 'توسعه', 'دهندگان', ...",'' برنامه نویسی اندروید''
3,"['Microsoft', 'Research', 'Open', 'Data', 'ریپ...",'' داده کاوی''
4,"['کامپیوتر\u200cها', 'بزرگی', 'زندگی', 'تشکیل'...",'' کامپیوتر''


## FreqDist

In [ ]:
from nltk import FreqDist

In [ ]:
all_tokens = []
for body in data['body']:
  all_tokens+= re.sub('\'|\"|\[|\]','',body).split(',')

In [ ]:
fd_all = FreqDist(all_tokens)
fd_all[0]

0

In [ ]:
keys = pd.DataFrame([(item,fd_all[item]) for item in fd_all])
keys.columns = ['word','freq']
keys = keys.sort_values(by=['freq'],ascending=False)
keys = keys.reset_index(drop=True)
keys.to_csv('../Data/FreqDist_sorted.csv', index=False)

In [ ]:
keys.head()

,word,freq
0,کار,112636
1,برنامه,65916
2,قرار,61977
3,سایت,51340
4,می\u200cتوانید,40276
